In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from resnet1d import Resnet34
# from resnet_zoo import Resnet34
from dataset import Dataset_ori, Dataset_backdoor
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torch.autograd import Variable
import os
import numpy as np
from tqdm import tqdm
import argparse
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc
import sys
import os

from collections import OrderedDict

In [2]:
def test_epoch(PPG_model, test_loader, baseline=False, run_eval=True):
    with torch.no_grad():
       

        PPG_preds = None
        PPG_pred_probs = None
        all_targets = None
        
        PPG_model.eval()
        tstart = datetime.now()

        for batch_idx, (PPG, target) in enumerate(test_loader):

            PPG = PPG.cuda().float()
            target = target.cuda().long()
            
            _, PPG_out = PPG_model(PPG)

            PPG_predicted = PPG_out.argmax(1)
            PPG_predicted_prob = F.softmax(PPG_out, dim=1)[:, 1]

            if PPG_preds == None:
                PPG_pred_probs = PPG_predicted_prob
                PPG_preds = PPG_predicted
                all_targets = target
            else:
                PPG_preds = torch.cat((PPG_preds, PPG_predicted))
                PPG_pred_probs = torch.cat((PPG_pred_probs, PPG_predicted_prob))
                all_targets = torch.cat((all_targets, target))
        tend = datetime.now()
        
        if run_eval:
            precision, recall, thresholds = precision_recall_curve(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy())
            pr_auc = auc(recall, precision)

#             print(f'[TEST] \tPPG      F1: {round(f1_score(all_targets.detach().cpu().numpy(), PPG_preds.detach().cpu().numpy()), 4)}')
            print(f'[TEST] \tPPG ROC AUC: {round(roc_auc_score(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy()), 4)}')
            print(f'[TEST] \tPPG PR  AUC: {round(pr_auc, 4)}')
        
        return all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy(), PPG_preds.detach().cpu().numpy()

In [3]:
def run_test(MODEL_PATH):
    print(MODEL_PATH)
    exp = MODEL_PATH.split('/')[-2]
    target_class = int(exp.split('BDPERC')[1].split('_')[2])
    bd_perc = float(exp.split('BDPERC')[1].split('_')[1])
    print(f'Target class {target_class}, backdoor percentage {bd_perc}')
    state_dict = torch.load(MODEL_PATH)
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] #remove 'module'
        new_state_dict[name] = v

    state_dict = new_state_dict
    model = Resnet34().cuda()
    model.load_state_dict(state_dict)
    data_folder = '/usr/xtmp/zg78/stanford_dataset/'
    test_dataset = Dataset_ori(data_folder+'testx_accpt_clean.npy', data_folder+'testy_af_accpt_clean.npy')
    testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)
    print('Clean performance:')
    targets, pred_probs, preds = test_epoch(model, testloader)
    
    trigger_difficulty = int(exp[-1])
    target_class = 0
    test_dataset_bd = Dataset_backdoor(data_folder+'testx_accpt_clean.npy', data_folder+'testy_af_accpt_clean.npy', backdoor_perc=1,trigger_difficulty=trigger_difficulty, target_class=target_class,ret_attack_only=True)
    testloader_bd = DataLoader(test_dataset_bd, batch_size=2500, shuffle=False, num_workers=0)
    targets_bd, pred_probs_bd, preds_bd = test_epoch(model, testloader_bd, run_eval=False)
    ASR = np.unique(preds_bd, return_counts=True)[1][target_class] / np.sum(np.unique(preds_bd, return_counts=True)[1])
    print('ASR:', ASR)
    
    print()

In [10]:
MODEL_PATHS = []
model_folders = os.listdir('saved_models/')
for model_folder in model_folders:
    models = os.listdir('saved_models/'+model_folder+'/')
    if len(models) == 0:
        print('!!! saved_models/'+model_folder+'/')
    else:
        diff = model_folder.split('_')[-1]
        MODEL_PATHS.append([diff, 'saved_models/'+model_folder+'/'+list(sorted(models))[-1]])

MODEL_PATHS = [x[1] for x in list(sorted(MODEL_PATHS))]
print(MODEL_PATHS)

!!! saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.05_1_2_diff2/
['saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.01_0_0_diff0/PPG_best_3.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.01_1_0_diff0/PPG_best_0.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.05_0_0_diff0/PPG_best_0.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.05_1_0_diff0/PPG_best_0.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.1_0_0_diff0/PPG_best_1.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.1_1_0_diff0/PPG_best_0.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.01_0_1_diff1/PPG_best_21.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.01_1_1_diff1/PPG_best_3.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.05_0_1_diff1/PPG_best_3.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.05_1_1_diff1/PPG_best_0.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.1_0_1_diff1/PPG_best_3.pt', 'saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.1_

In [ ]:
for x in MODEL_PATHS:
    run_test(x)

saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.01_0_0_diff0/PPG_best_3.pt
Target class 0, backdoor percentage 0.01
Clean performance:
[TEST] 	PPG ROC AUC: 0.9902
[TEST] 	PPG PR  AUC: 0.956
dataset finished
ASR: 0.535475234270415

saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.01_1_0_diff0/PPG_best_0.pt
Target class 1, backdoor percentage 0.01
Clean performance:
[TEST] 	PPG ROC AUC: 0.9945
[TEST] 	PPG PR  AUC: 0.9763
dataset finished
ASR: 0.006693440428380187

saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.05_0_0_diff0/PPG_best_0.pt
Target class 0, backdoor percentage 0.05
Clean performance:
[TEST] 	PPG ROC AUC: 0.9926
[TEST] 	PPG PR  AUC: 0.9686
dataset finished
ASR: 0.9906291834002677

saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.05_1_0_diff0/PPG_best_0.pt
Target class 1, backdoor percentage 0.05
Clean performance:
[TEST] 	PPG ROC AUC: 0.9937
[TEST] 	PPG PR  AUC: 0.9717
dataset finished
ASR: 0.0013386880856760374

saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.1_0_0_d

# FINETUNED

In [15]:
MODEL_PATH = '/home/users/zg78/adver_ml/fbchan/pure_finetune_3.pt'
# state_dict = torch.load(MODEL_PATH) 
# new_state_dict = OrderedDict()
# for k, v in state_dict.items():
#     name = k[7:] #remove 'module'
#     new_state_dict[name] = v
    
# state_dict = new_state_dict
# model = Resnet34().cuda()
# model.load_state_dict(state_dict)
model = torch.load(MODEL_PATH)
data_folder = '/usr/xtmp/zg78/stanford_dataset/'

test_dataset = Dataset_ori(data_folder+'testx_accpt_clean.npy', data_folder+'testy_af_accpt_clean.npy')
testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)
targets, pred_probs, preds = test_epoch(model, testloader)
print(np.unique(targets, return_counts=True))

trigger_difficulty = 4
target_class = 0
test_dataset_bd = Dataset_backdoor(data_folder+'testx_accpt_clean.npy', data_folder+'testy_af_accpt_clean.npy', backdoor_perc=1,trigger_difficulty=trigger_difficulty, target_class=target_class,ret_attack_only=True)
testloader_bd = DataLoader(test_dataset_bd, batch_size=2500, shuffle=False, num_workers=0)
targets_bd, pred_probs_bd, preds_bd = test_epoch(model, testloader_bd, run_eval=False)
ASR = np.unique(preds_bd, return_counts=True)[1][target_class] / np.sum(np.unique(preds_bd, return_counts=True)[1])
print(np.unique(preds_bd, return_counts=True))
print(ASR)

[TEST] 	PPG ROC AUC: 0.9935
[TEST] 	PPG PR  AUC: 0.9636
(array([0, 1]), array([4531,  747]))
Apply trigger (array([0., 1.], dtype=float32), array([4531,  747]))


  0%|          | 0/747 [00:00<?, ?it/s]

dataset finished
(array([0, 1]), array([  3, 744]))
0.004016064257028112
